# PyTorch `tensor.backward()` fucntion
<font color="steelblue" size="4">

1. This post examines some `tensor.backward()` function examples about the `autograd (Automatic Differentiation)` package of PyTorch.
2. As you already know, if you want to compute all the derivatives of a tensor, you can call `backward()` on it. (`tensor.backward()`)
3. <font color="red">The `torch.tensor.backward()` relies on the autograd function `torch.autograd.backward()` that computes the `sum of gradients (without returning it) of given tensors with respect to the graph leaves（all leaf nodes/tensor）`.
    - 例如在下图中，当`loss.backward()`之后，可以得到
        1. `x.grad`
        2. `W.grad`
        3. `b.grad`
        4. `y.grad`
        5. `z.grad` is not a leaf tensor
</font>

![computation_graph](./pics/computational_graph.png)

</font>

# 1. A first Example

## 1.1. Example 的描述
<font color="steelblue" size="4">

1. Give a matrix $x$,
$$\begin{aligned}
x :=
\begin{bmatrix}
x_1 & x_2 \\
x_3 & x_4
\end{bmatrix} = 
\begin{bmatrix}
1 & 1 \\
1 & 1
\end{bmatrix}
\end{aligned}$$
2. and another matrix y is defined as
$$\begin{aligned}
y := 
x + 2 = 
\begin{bmatrix}
x_1 + 2  &  x_2 + 2 \\
x_3 + 2  &  x_4 + 2
\end{bmatrix} = 
\begin{bmatrix}
3 & 3 \\
3 & 3 
\end{bmatrix}
\end{aligned}$$
3. We define z as:
    - <font color="red">Note: `*` -- entry-wise multiplication</font>
$$\begin{aligned}
z = y * y * 3 = 
3 * 
\begin{bmatrix}
y_1 & y_2 \\ 
y_3 & y_4
\end{bmatrix} * 
\begin{bmatrix}
y_1 & y_2 \\ 
y_3 & y_4
\end{bmatrix} = 
\begin{bmatrix}
3y_1^2 & 3y_2^2 \\ 
3y_3^2 & 3y_4^2
\end{bmatrix}
\end{aligned}$$
4. Finally, we define out as:
$$\begin{aligned}
out = \frac{1}{4}(3y_1^2 + 3y_2^2 + 3y_3^2 + 3y_4^2)
\end{aligned}$$
5. The value of derivative of out with respect with $x_2$
$$\begin{aligned}
\frac{\partial out}{\partial x_2}
&= \frac{\partial}{\partial x_2} \left( \frac{1}{4}(3y_1^2+3y_2^2+3y_3^2+3y_4^2) \right) \\
&= 0 + \frac{3}{4}\frac{\partial}{\partial x_2}y_2^2 + 0 + 0 \\
&= \frac{3}{4}\frac{\partial}{\partial x_2}(x_2 + 2)^2 \\
&= \frac{3}{2}(x_2 + 2)
\end{aligned}$$

</font>


<font color="coral" size="4">

Note
----
1. $out$ contains `a single real value`(`scaler`). This value is the result of `scalar function`(In the case, the `mean` function).

</font>

## 1.2. Code for example

<font color="coral" size="4">

Note
----
1. The `grad` attribute of `x` is None by default and becomes a tensor the first time a call to `out.backward()` computes gradients for self($\frac{\partial out}{\partial x}$).
2. The `grad` attribute will then contain the gradients computed and future calls to `backward()` will accumulate (add) gradients into it. 
3. Alternative option for `tensor.backward()`:
    - `torch.autograd.grad(outputs=out, inputs=x)`

</font>

In [21]:
import torch

### Part I. device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(device)

### Part II. example code
## Step 1. define how to calculate
x = torch.ones(
            (2, 2),
            device=device,
            requires_grad=True,
            dtype=torch.float32,
            )

# y.grad_fn = AddBackward
y = x + 2
# z.grad_fn = MultiBackward
z = y * y  * 3  # element-wise multiplcation
# out.grad_fn = MeanBackward
output = z.mean()
print("out (a scaler) = ", output.item())

## Step 2. calculate the `derivatives of out with respect to x`: dout/dx
output.backward()
print("dout/dx = ", x.grad)

cpu
out (a scaler) =  27.0
dout/dx =  tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


## 1.3. `x.grad` will accumulate if not `optimizer.zero_grad()` in training loop
<font color="coral" size="4">

Note
----
1. The `grad` attribute will then contain the gradients computed and future calls to `backward()` will accumulate (add) gradients into it. 

</font>

In [22]:
# Step 1. init torch.tensor x
x = torch.ones(
            (2, 2),
            device=device,
            requires_grad=True,
            dtype=torch.float32,
            )

def calculate_output(x: torch.tensor):
    y = x + 2
    z = y * y * 3   # element-wise multiplication
    output = z.mean()
    output.backward()


# Step 2. first time
calculate_output(x=x)
print("First:\n\tdout/dx = ", x.grad)
print("")


# Step 3. second time
calculate_output(x=x)
print("Second:\n\tdout/dx = ", x.grad)

First:
	dout/dx =  tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

Second:
	dout/dx =  tensor([[9., 9.],
        [9., 9.]])


# 2. A neural networks example
<font color="steelblue" size="4">

1. <font color="red">Neural networks use `backpropagation algorithm`: neural network parameters (`model weights`) are adjusted according to `the gradient of the loss function with respect to the given parameters`.
    - 我们需要求的是 `损失函数相对于权重的导数`</font>
2. PyTorch has `torch.autograd` as built-in engine to compute those gradients.
3. The engine supports `automatic computation of gradients` for any `computational graph`.
4. Consider the simplest one-layer neural network, with input $x$, parameters $W$ and $b$, and some `loss function`.

</font>

## 2.1. Code for simplest neural network

In [32]:
### Step 1. Data Preparation
# input tensor
x = torch.ones(
                8,
                dtype=torch.float32,
                requires_grad=True,
                device=device,
                )
# output tensor
y = torch.zeros(
                10,
                dtype=torch.float32,
                requires_grad=True,
                device=device,
                )
# weights
W = torch.randn(
                (8, 10),
                dtype=torch.float32,
                requires_grad=True,
                device=device,
                )
# bias vector
b = torch.randn(10,
                dtype=torch.float32,
                requires_grad=True,
                device=device,
                ) 


### Step 2. Calculate
z = torch.matmul(x, W) + b # output
z.requires_grad_(True)
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

### Step 3. 
loss.backward()
print(x.grad)
print(W.grad)
print(b.grad)
print(y.grad)

tensor([-0.0631, -0.0755, -0.1221,  0.2267,  0.2879,  0.0203,  0.2067,  0.1933])
tensor([[0.0113, 0.0900, 0.0055, 0.0827, 0.0002, 0.0205, 0.0006, 0.0100, 0.0131,
         0.0980],
        [0.0113, 0.0900, 0.0055, 0.0827, 0.0002, 0.0205, 0.0006, 0.0100, 0.0131,
         0.0980],
        [0.0113, 0.0900, 0.0055, 0.0827, 0.0002, 0.0205, 0.0006, 0.0100, 0.0131,
         0.0980],
        [0.0113, 0.0900, 0.0055, 0.0827, 0.0002, 0.0205, 0.0006, 0.0100, 0.0131,
         0.0980],
        [0.0113, 0.0900, 0.0055, 0.0827, 0.0002, 0.0205, 0.0006, 0.0100, 0.0131,
         0.0980],
        [0.0113, 0.0900, 0.0055, 0.0827, 0.0002, 0.0205, 0.0006, 0.0100, 0.0131,
         0.0980],
        [0.0113, 0.0900, 0.0055, 0.0827, 0.0002, 0.0205, 0.0006, 0.0100, 0.0131,
         0.0980],
        [0.0113, 0.0900, 0.0055, 0.0827, 0.0002, 0.0205, 0.0006, 0.0100, 0.0131,
         0.0980]])
tensor([0.0113, 0.0900, 0.0055, 0.0827, 0.0002, 0.0205, 0.0006, 0.0100, 0.0131,
        0.0980])
None
tensor([ 0.2058, -0.2192

/tmp/ipykernel_1776/1324917252.py:41: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  print(z.grad)


## 2.2. Computatioal Graph
<font color="steelblue" size="4">

1. Some node in graph:
    - $x$: input
    - $W$: model weight
    - $b$: model bias
    - $y$: real result
    - $z$: result predicted by model (is not a leaf node/tensor)
2. We can only obtain the grad properties for the <font color="red">leaf nodes/tensor</font> of the computational graph which have <font color="red">requires_grad</font> property set to True. Calling grad on non-leaf nodes will elicit a warning message:
   ```bash
   UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward().
   ```
</font>

![computation_graph](./pics/computational_graph.png)

<font color="coral" size="4">

Note
----
1. Note that $loss$ is a scalar output. Applying `loss.backward()` directly on $loss$ (with no arguments) is not a problem because `loss represents a unique output and it is unambiguous to take its derivatives with respect to each variable in` $x$ / $W$ / $b$ / $y$. 
2. The situation changes when trying to call `backward()` on a `non-scalar output`? 
    - For example, consider the 10-entries tensor $z$. When calling `z.backward()` on it, what do you expect `x.grad` to be? We will address this problem in the following sections.

</font>

# 3. Vector-Jacobian product
<font color="steelblue" size="4">

1. In general, `torch.autograd` is an engine for computing `vector-Jacobian products`
   1. If `computational graph` is like below:
      - $y = Y(x)$
      - $l = L(y)$
![cg_1](./pics/cg_1.png)
   2. According to `rule of chain`:
      - $j$ 最大为 $y$ 所在层神经元的个数
      - $i$ 最大为 $x$ 所在层神经元的个数
      - <font color="red">$l$ is a `scalar`</font>, so $\frac{\partial l}{\partial y_i}$ 的所有项组成一个 `vector` -- $v$
      - $\frac{\partial y_i}{\partial x_j}$ 的所有项组成一个 `Jacobian matrix` -- $J$
$$\begin{aligned}
\frac{\partial l}{\partial x_j}=
\frac{\partial l}{\partial y_i} \cdot \frac{\partial y_i}{\partial x_j}
\end{aligned}$$
2. the product $J^T\cdot\vec{v}$ where $\vec{v}$ is any vector and
$$\begin{aligned}
J^T \cdot v = 
\begin{bmatrix}
\frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_n}{\partial x_1} \\ 
\vdots & \ddots & \vdots \\
\frac{\partial y_1}{\partial x_m} & \cdots & \frac{\partial y_n}{\partial x_m}
\end{bmatrix}
\begin{bmatrix}
\frac{\partial l}{\partial y_1} \\
\vdots \\
\frac{\partial l}{\partial y_m}
\end{bmatrix} = 
\begin{bmatrix}
\frac{\partial l}{\partial x_1} \\
\vdots \\
\frac{\partial l}{\partial x_n}
\end{bmatrix}
\end{aligned}$$

</font>

## 3.1. Demo 1
### 3.1.1. Computational graph
<font color="steelblue" size="4">

1. $y_i = x_i + 2$
2. $z_i = 3y_i^2$
3. $out = mean(z)$

</font>

![cg_2](./pics/cg_2.png)

### 3.1.2. Procedure
<font color="steelblue" size="4">

1. According to `rule of chain`:
    - 因为 $out$ 是一个标量，因此所有的 $\frac{\partial out}{\partial x}$ 项构成一个 `vector` -- $v$
    - 所有的 $\frac{\partial z}{\partial y}$ 项构成 `Jacobian Matrix` -- $J_1$
    - 所有的 $\frac{\partial y}{\partial x}$ 项构成 `Jacobian Matrix` -- $J_2$
$$\begin{aligned}
\frac{\partial out}{\partial x} = 
\frac{\partial out}{\partial z} \cdot
\frac{\partial z}{\partial y} \cdot
\frac{\partial y}{\partial x}
\end{aligned}$$
2. 求 $J_1$
$$\begin{aligned}
J_1^T = 
\begin{bmatrix}
\frac{\partial z_1}{\partial y_1} & \cdots & \frac{\partial z_4}{\partial y_1} \\
\vdots & \ddots & \vdots \\
\frac{\partial z_1}{\partial y_4} & \cdots & \frac{\partial z_4}{\partial y_4}
\end{bmatrix} = 
\begin{bmatrix}
6 & 0 & 0 & 0 \\
0 & 6 & 0 & 0 \\
0 & 0 & 6 & 0 \\
0 & 0 & 0 & 6 \\
\end{bmatrix}
\end{aligned}$$
3. 求 $J_2$
$$\begin{aligned}
J_2^T = 
\begin{bmatrix}
\frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_4}{\partial x_1} \\
\vdots & \ddots & \vdots \\
\frac{\partial y_1}{\partial x_4} & \cdots & \frac{\partial z_4}{\partial y_4}
\end{bmatrix} = 
\begin{bmatrix}
\frac{1}{4} & 0 & 0 & 0 \\
0 & \frac{1}{4} & 0 & 0 \\
0 & 0 & \frac{1}{4} & 0 \\
0 & 0 & 0 & \frac{1}{4} \\
\end{bmatrix}
\end{aligned}$$
4. 求 $v$
$$\begin{aligned}
v = 
\begin{bmatrix}
\frac{\partial l}{\partial z_1} \\
\frac{\partial l}{\partial z_2} \\
\frac{\partial l}{\partial z_3} \\
\frac{\partial l}{\partial z_4} \\
\end{bmatrix}
\end{aligned}$$
5. 求 $J_1^TJ_2^Tv$
   ```python
   # 代码求 J_1^TJ_2^Tv
   out.backward()
   result = x.grad()
   ```
$$\begin{aligned}
J_1^TJ_2^Tv = 
\begin{bmatrix}
\frac{3}{2}\frac{\partial l}{\partial z_1} \\
\frac{3}{2}\frac{\partial l}{\partial z_2} \\
\frac{3}{2}\frac{\partial l}{\partial z_3} \\
\frac{3}{2}\frac{\partial l}{\partial z_4} \\
\end{bmatrix}
\end{aligned}$$

</font>

## 3.2. Demo 2

In [34]:
import torch

x = torch.rand(
            3,
            device=device,
            requires_grad=True,
            dtype=torch.float32,
            )
y = x + 2


# y.backward()
# Abort Error : 
#       RuntimeError: grad can be implicitly created only for scalar outputs

# v: `vector` in `Jacobaian Matrix * vector`
v = torch.rand(
            3,
            device=device,
            requires_grad=True,
            dtype=torch.float32,
            )


y.backward(v)
print(x.grad)

tensor([0.6912, 0.3989, 0.8603])


## 3.3. Demo 3
<font color="steelblue" size="4">

1. Let $x=[x_1, x_2]$ and define $y$ as 
$$\begin{aligned}
[y_1, y_2, y_3] := [x_1^2, x_1^2+5x_2^2, 3x_2]
\end{aligned}$$
2. In this case, `tansposed Jacobina Matrix` is 
$$\begin{aligned}
\begin{bmatrix}
\frac{\partial y_1}{\partial x_1} & \frac{\partial y_2}{\partial x_1} & \frac{\partial y_3}{\partial x_1} \\
\frac{\partial y_1}{\partial x_2} & \frac{\partial y_2}{\partial x_2} & \frac{\partial y_3}{\partial x_2} \\
\end{bmatrix} = 
\begin{bmatrix}
2x_1 & 2x_1 & 0 \\
0 & 10x_2 & 3
\end{bmatrix}
\end{aligned}$$
3. If $x=[1, 2]$, choose vector $v = [1, 1, 1]$
$$\begin{aligned}
\begin{bmatrix}
2 & 2 & 0 \\
0 & 20 & 3
\end{bmatrix}
\begin{bmatrix}
1 \\
1 \\
1
\end{bmatrix} = 
\begin{bmatrix}
4 \\
23 
\end{bmatrix}
\end{aligned}$$

</font>

In [36]:
x = torch.tensor(
            [1, 2],
            device=device,
            requires_grad=True,
            dtype=torch.float32,
            )

y = torch.empty(3)
y[0] = x[0]**2
y[1] = x[0]**2 + 5*x[1]**2
y[2] = 3*x[1]
print("y = ", y)

v = torch.ones(
            3, 
            device=device,
            requires_grad=True,
            dtype=torch.float32,
            )

y.backward(v)
print("x.grad = ", x.grad)

y =  tensor([ 1., 21.,  6.], grad_fn=<CopySlices>)
x.grad =  tensor([ 4., 23.])


## 3.4. The general case
<font color="steelblue" size="4">

1. We have seen - and it is also shown on the `official autograd page` -- that if you have a function $y = f(x)$
2. But what happens when v  is not a simple vector? See `Demo 1.`

</font>